## Домашнее творческое задание по дисциплине "Технологии обработки больших данных" (часть 2)

Выполнил: студент группы __ПМ20-4 Кондаков Илья Владимирович__ и студент группы __ПМ20-4 Радайкин Артём Альбертович__ 

Вариант: __50__

__Предметная область__ : Climate Change: Earth Surface Temperature Data

__Источник данных__: https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

__В задачах, связанных с пакетом `dask`, явно указано, какой коллекцией вы должны пользоваться при решении задачи. Все операции вычислений должны проводиться  над этой коллекцией и средствами пакета `dask`, если в задании не сказано обратного.__ 



In [1]:
import pandas as pd
import numpy as np
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import re
import multiprocessing
import sqlite3
import json

__Задание 1__. Придумайте несколько варианта посчета количества количества городов для каждой из стран (GlobalLandTemperaturesByCity.csv). Замерьте время выполнения этих решений при помощи time и timeit.

In [2]:
df = pd.read_csv('GlobalLandTemperaturesByCity.csv', encoding='UTF-8', parse_dates=['dt'])

In [3]:
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [4]:
%%time
df[['Country', 'City']].drop_duplicates().groupby('Country').count()

CPU times: total: 1.66 s
Wall time: 1.69 s


,City
Country,
Afghanistan,8
Albania,3
Algeria,5
Angola,6
Argentina,32
...,...
Venezuela,40
Vietnam,30
Yemen,2


In [5]:
%%timeit
df[['Country', 'City']].drop_duplicates().groupby('Country').count()

1.64 s ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%time

dict_ = {}

def cnt(row):
    global dict_
    country = row['Country']
    city = row['City']
    
    if country in dict_.keys():
        if city not in dict_[country]:
            dict_[country].append(city)
    else:
        dict_[country] = [city]

df[['Country', 'City']].apply(cnt, axis=1) 

for key in dict_.keys():
    dict_[key] = len(dict_[key])
    
pd.DataFrame.from_dict(dict_, orient='index', columns=['City']).sort_index()

CPU times: total: 2min 58s
Wall time: 3min


,City
Afghanistan,8
Albania,3
Algeria,5
Angola,6
Argentina,32
...,...
Venezuela,40
Vietnam,30
Yemen,2
Zambia,8


In [8]:
def cnt(row):
    global dict_
    country = row['Country']
    city = row['City']
    
    if country in dict_.keys():
        if city not in dict_[country]:
            dict_[country].append(city)
    else:
        dict_[country] = [city]

In [9]:
%%timeit

dict_ = {}

df[['Country', 'City']].apply(cnt, axis=1) 

for key in dict_.keys():
    dict_[key] = len(dict_[key])
    
pd.DataFrame.from_dict(dict_, orient='index', columns=['City']).sort_index()

TypeError: argument of type 'int' is not iterable

__Задание 2__. На основе данных из GlobalLandTemperaturesByMajorCity.csv создайте таблицу в БД SQLite. Продемонстрируйте работу функции pd.read_sql_query, выполнив SQL-запрос с группировкой

In [62]:
df_tem_by_majorcity = pd.read_csv('GlobalLandTemperaturesByMajorCity.csv', encoding='UTF-8', parse_dates=['dt'])

In [64]:
df_tem_by_majorcity.head(2)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1849-01-01,26.704,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W
1,1849-02-01,27.434,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W


In [65]:
df_tem_by_majorcity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239177 entries, 0 to 239176
Data columns (total 7 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   dt                             239177 non-null  datetime64[ns]
 1   AverageTemperature             228175 non-null  float64       
 2   AverageTemperatureUncertainty  228175 non-null  float64       
 3   City                           239177 non-null  object        
 4   Country                        239177 non-null  object        
 5   Latitude                       239177 non-null  object        
 6   Longitude                      239177 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 12.8+ MB


In [66]:
con = sqlite3.connect("dtz.sqlite")

In [67]:
cur = con.cursor()

In [71]:
df_tem_by_majorcity.to_sql('Temp_majorcity', con=con, if_exists='append', index=False)

239177

In [72]:
cur.execute("SELECT * FROM Temp_majorcity;")
all_results = cur.fetchmany(2)
print(all_results)

[('1849-01-01 00:00:00', 26.704, 1.435, 'Abidjan', "Côte D'Ivoire", '5.63N', '3.23W'), ('1849-02-01 00:00:00', 27.434, 1.362, 'Abidjan', "Côte D'Ivoire", '5.63N', '3.23W')]


In [82]:
query = '''
        SELECT Country, AVG(AverageTemperature) as avg_temp
        FROM Temp_majorcity
        GROUP BY Country
        HAVING avg_temp > 26
        '''
pd.read_sql_query(query, con=con)

,Country,avg_temp
0,Burma,26.735193
1,Côte D'Ivoire,26.163737
2,Indonesia,26.659057
3,Nigeria,26.361323
4,Philippines,26.448334
5,Saudi Arabia,26.427309
6,Singapore,26.523103
7,Somalia,27.151964
8,Sudan,29.081291
9,Thailand,27.164733


__Задание 3__. Используя multiprocessing.Pool, продемонстрируйте, как можно применить параллельные вычисления для нахождения века с максимальной температурой (GlobalLandTemperaturesByCity.csv).

In [2]:
shape = 8599212
K  = 8
chunk = 8599212 // K

In [5]:
%%file max_temp_with_centure.py

def max_temp_with_centure(df):
    max_temp = df.AverageTemperature.max()
    year = df.loc[df.AverageTemperature.idxmax()]['dt'][:4]
    if year[2:4] == '00':
        centure = int(year[:2])
    else:
        centure = int(year[:2]) + 1
               
    return [centure, max_temp]

Overwriting max_temp_with_centure.py


In [6]:
from max_temp_with_centure import max_temp_with_centure

In [7]:
with multiprocessing.Pool(processes=K) as pool:
    res = pool.map(max_temp_with_centure, [df for df in pd.read_csv('GlobalLandTemperaturesByCity.csv', chunksize=chunk)])
res

[[21, 39.15600000000001],
 [21, 38.049],
 [20, 36.889],
 [21, 38.531],
 [21, 39.15600000000001],
 [21, 37.859],
 [21, 37.938],
 [18, 39.651],
 [21, 18.775]]

In [8]:
max(res, key=lambda x: x[1])

[18, 39.651]

__Задание 4__. Используя dask.array (GlobalLandTemperaturesByCity.csv), соберите статистику по столбцу AverageTemperature с учетом неопределенности из столбца AverageTemperatureUncertainty


In [11]:
df_gl_by_city = pd.read_csv('GlobalLandTemperaturesByCity.csv', parse_dates = ['dt'])
df_gl_by_city.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [14]:
da_avg_temp = da.from_array(df_gl_by_city[['AverageTemperature', 'AverageTemperatureUncertainty']].values)
da_avg_temp

dask.array<array, shape=(8599212, 2), dtype=float64, chunksize=(8388608, 2), chunktype=numpy.ndarray>

In [17]:
da_avg_temp[:5, 0].compute()

array([6.068,   nan,   nan,   nan,   nan])

In [23]:
da_avg_temp[:, 0].compute()

array([ 6.068,    nan,    nan, ..., 18.775, 18.025,    nan])

In [34]:
import dask
dask.__version__

'2022.12.0'

In [59]:
res_of_conf_interval = da_avg_temp[(-da_avg_temp[:, 1] < da_avg_temp[:, 0]) & ( da_avg_temp[:, 0] < da_avg_temp[:, 1])][:, 0]

In [62]:
mean = res_of_conf_interval.mean()
max_ = res_of_conf_interval.max()
min_ = res_of_conf_interval.min()
std = res_of_conf_interval.std()

In [63]:
da.compute(mean, max_, min_, std)

(1.88870670810785, 15.396, -11.636, 2.725413768849465)

__Задание 5__. Используя dask.bag, найдите максимальное значение средней температуры на каждый год (GlobalLandTemperaturesByCity.csv) при помощи foldby

In [67]:
df_gl_temp = pd.read_csv('GlobalLandTemperaturesByCity.csv')

In [65]:
db.read_text('GlobalLandTemperaturesByCity.csv').take(5)

('dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude\n',
 '1743-11-01,6.068,1.7369999999999999,Århus,Denmark,57.05N,10.33E\n',
 '1743-12-01,,,Århus,Denmark,57.05N,10.33E\n',
 '1744-01-01,,,Århus,Denmark,57.05N,10.33E\n',
 '1744-02-01,,,Århus,Denmark,57.05N,10.33E\n')

In [122]:
df1 = df_gl_temp[['dt', 'AverageTemperature']].dropna().to_dict(orient='records')

In [123]:
with open('GlobalLandTemperaturesByCity.json', 'w') as outfile:
    for row in df1:
        outfile.write(str(row).replace("'", '"') + '\n')

In [124]:
bag = db.read_text('GlobalLandTemperaturesByCity.json').map(json.loads)

({'dt': '1743-11-01', 'AverageTemperature': 6.068},
 {'dt': '1744-04-01', 'AverageTemperature': 5.787999999999999},
 {'dt': '1744-05-01', 'AverageTemperature': 10.644},
 {'dt': '1744-06-01', 'AverageTemperature': 14.050999999999998},
 {'dt': '1744-07-01', 'AverageTemperature': 16.082})

In [125]:
def binop(l, r):
    if l:
        return max((l, r), key=lambda x: x['AverageTemperature'])
    return r

bag.foldby(lambda p: p['dt'][:4], binop).compute()

[('1743', {'dt': '1743-11-01', 'AverageTemperature': 18.722}),
 ('1744', {'dt': '1744-07-01', 'AverageTemperature': 27.496}),
 ('1745', {'dt': '1745-04-01', 'AverageTemperature': 21.63}),
 ('1750', {'dt': '1750-07-01', 'AverageTemperature': 29.613000000000003}),
 ('1751', {'dt': '1751-07-01', 'AverageTemperature': 28.611}),
 ('1752', {'dt': '1752-10-01', 'AverageTemperature': 22.244}),
 ('1753', {'dt': '1753-07-01', 'AverageTemperature': 36.397}),
 ('1754', {'dt': '1754-07-01', 'AverageTemperature': 35.626}),
 ('1755', {'dt': '1755-07-01', 'AverageTemperature': 35.36799999999999}),
 ('1756', {'dt': '1756-07-01', 'AverageTemperature': 34.864000000000004}),
 ('1757', {'dt': '1757-07-01', 'AverageTemperature': 38.414}),
 ('1758', {'dt': '1758-07-01', 'AverageTemperature': 34.392}),
 ('1759', {'dt': '1759-08-01', 'AverageTemperature': 34.898}),
 ('1760', {'dt': '1760-08-01', 'AverageTemperature': 34.431999999999995}),
 ('1761', {'dt': '1761-07-01', 'AverageTemperature': 39.651}),
 ('1762',

__Задание 6__. Используя dask.dataFrame (GlobalLandTemperaturesByCity.csм), найдите координаты самой северо-восточной точки из датасета

In [7]:
df_gl_temp = pd.read_csv('GlobalLandTemperaturesByCity.csv', parse_dates=['dt'])

In [6]:
df_gl_temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [8]:
df_gl_temp.Latitude.apply(lambda x: x[-1]).value_counts()

N    7323138
S    1276074
Name: Latitude, dtype: int64

In [9]:
df_gl_temp.Longitude.apply(lambda x: x[-1]).value_counts()

E    6100003
W    2499209
Name: Longitude, dtype: int64

In [43]:
# нужны макс NE сумма
df = dd.read_csv("GlobalLandTemperaturesByCity.csv",
                 parse_dates=['dt'],
                 assume_missing=True)

In [44]:
coords = df[['Latitude', 'Longitude']].drop_duplicates()

In [32]:
coords.head(5)

,Latitude,Longitude
0,57.05N,10.33E
3239,40.99N,27.69E
6478,40.99N,34.08E
9607,50.63N,82.39E
11925,44.20N,87.20E


In [45]:
df.loc[coords.apply(lambda x: float(x.Latitude[:-1]) + float(x.Longitude[:-1]) \
         if (x.Latitude[-1] == 'N') and (x.Longitude[-1] == 'E') else 0, 
         axis=1,  
         meta=('sum_of_coords', 'float')).idxmax()].compute()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
437503,1980-05-01,22.562,0.489,Asgabat,Turkmenistan,37.78N,58.98E
437503,1895-04-01,4.910,0.474,Chernihiv,Ukraine,52.24N,31.53E
437503,2005-07-01,17.551,0.290,Glazov,Russia,58.66N,52.31E
437503,1853-01-01,0.464,1.860,Kashan,Iran,34.56N,50.59E
437503,1880-07-01,26.308,0.619,Melaka,Malaysia,2.41N,102.05E
437503,1839-08-01,12.157,1.811,Petropavlovsk Kamchatskiy,Russia,53.84N,159.55E
437503,1910-09-01,24.633,0.851,Shaki,Nigeria,8.84N,4.05E
437503,1791-12-01,-0.405,2.475,Vanadzor,Armenia,40.99N,44.73E


In [51]:
coords.loc[coords.apply(lambda x: float(x.Latitude[:-1]) + float(x.Longitude[:-1]) \
         if (x.Latitude[-1] == 'N') and (x.Longitude[-1] == 'E') else 0, 
         axis=1,  
         meta=('sum_of_coords', 'float')).idxmax()].compute()

,Latitude,Longitude
437503,53.84N,159.55E


In [50]:
df.loc[437503].compute()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
437503,1980-05-01,22.562,0.489,Asgabat,Turkmenistan,37.78N,58.98E
437503,1895-04-01,4.910,0.474,Chernihiv,Ukraine,52.24N,31.53E
437503,2005-07-01,17.551,0.290,Glazov,Russia,58.66N,52.31E
437503,1853-01-01,0.464,1.860,Kashan,Iran,34.56N,50.59E
437503,1880-07-01,26.308,0.619,Melaka,Malaysia,2.41N,102.05E
437503,1839-08-01,12.157,1.811,Petropavlovsk Kamchatskiy,Russia,53.84N,159.55E
437503,1910-09-01,24.633,0.851,Shaki,Nigeria,8.84N,4.05E
437503,1791-12-01,-0.405,2.475,Vanadzor,Armenia,40.99N,44.73E
